# 🌐 Import & Inspect SPOKE‑GeneLab Knowledge Graph in Neo4j

This notebook bulk‑imports the CSV node and relationship exports from the SPOKE‑GeneLab pipeline into a Neo4j graph database, then runs exploratory queries to validate schema, metadata, counts, and full‑text search functionality. It supports downloading into a Neo4j Desktop database or Neo4j Enterprise instance (see cell [5])

**Start the `spoke-genelab` Graph DBMS before you run this notebook!**

Author: Peter W. Rose, UC San Diego (pwrose.ucsd@gmail.com)

In [1]:
import os
import pandas as pd
from py2neo import Graph
import neo4j_utils
import neo4j_bulk_importer

In [2]:
pd.set_option('display.max_rows', None)  # Shows all rows
pd.set_option('display.max_columns', None) # Shows all columns
pd.set_option('display.max_colwidth', None)  # Shows full content of each cell

### Import the Knowledge Graph
CSV data and metadata files are uploaded into the Neo4j Graph database from the [kg](https://github.com/BaranziniLab/spoke_genelab/tree/main/kg) directory using the [kg-import](https://github.com/sbl-sdsc/kg-import) bulk upload scripts. For a description of the data organization and the specification of metadata [see](https://github.com/sbl-sdsc/kg-import/blob/main/README.md).

In [3]:
neo4j_bulk_importer.import_from_csv_to_neo4j_desktop(verbose=True) # Use for Neo4j Desktop
# neo4j_bulk_importer.import_from_csv_to_neo4j_enterprise(verbose=True) # Use for Neo4j Enterprise instance

drop_database: '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-11a50ea0-7f7c-49f9-b49b-00ffe185cc16/bin/cypher-shell' -d system -u neo4j -p neo4jdemo 'DROP DATABASE `spoke-genelab-v0.0.3` IF EXISTS;'



Executing:   0%|          | 0/85 [00:00<?, ?cell/s]

run_bulk_import: cd '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-11a50ea0-7f7c-49f9-b49b-00ffe185cc16/import'; '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-11a50ea0-7f7c-49f9-b49b-00ffe185cc16/bin/neo4j-admin' database import full spoke-genelab-v0.0.3 --overwrite-destination --skip-bad-relationships --skip-duplicate-nodes --multiline-fields --array-delimiter='|' @args.txt
Neo4j version: 5.24.0
Importing the contents of these files into /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-11a50ea0-7f7c-49f9-b49b-00ffe185cc16/data/databases/spoke-genelab-v0.0.3:
Nodes:
  [Assay]:
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-11a50ea0-7f7c-49f9-b49b-00ffe185cc16/import/header_Assay_n.csv
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-11a50ea0-7f7c-49f9-b49b-00ffe185cc16/im

### Connect to the local Neo4j Graph database

In [4]:
database = os.environ.get("NEO4J_DATABASE")
username = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")
stylesheet = os.environ.get("NEO4J_STYLESHEET")

graph = Graph("bolt://localhost:7687", name=database, user=username, password=password)

## Metadata <a class="anchor" id="Metadata"></a>

### Node metadata
spoke_genelab KG is a self-describing KG. The MetaNodes and MetaRelationships define the structure of the KG and the properties of nodes and relationships. The query below lists the nodes and their properties.

In [5]:
query = """
MATCH (n:MetaNode) RETURN n;
"""
df = graph.run(query).to_data_frame()
metadata = df["n"].tolist()
metadata = pd.DataFrame(metadata)
metadata.fillna("", inplace=True)
metadata

,nodeName,identifier,material_name_1,technology,material_name_2,measurement,factors_2,material_2,factors_1,material_1,name,material_id_1,material_id_2,organism,taxonomy,dist_to_feature,in_exon,in_promoter,chromosome,start,in_intron,end,end_date,flight_program,space_program,start_date,project_type,project_title
0,Anatomy,UBERON Ontology ID (string),,,,,,,,,,,,,,,,,,,,,,,,,,
1,Assay,GeneLab Data System GLDS-ID-MD5_hashcode(factors and materials) (string),"Prefered name from UEBERON for first material, e.g., cell type, tissue, organ (string)",Assay technology (string),"Prefered name from UEBERON for second material, e.g., cell type, tissue, organ (string)",Assay measurement type (string),Second factor(s) in transcriptional profiling (string[]),"Second material studied, e.g., cell type, tissue, organ (string)",First factor(s) in transcriptional profiling (string[]),"First material studied, e.g., cell type, tissue, organ (string)",Assay name (string),First material identifier (string),Second material identifier (string),,,,,,,,,,,,,,,
2,CellType,Cell Ontology ID (string),,,,,,,,,,,,,,,,,,,,,,,,,,
3,Gene,Gene ENTREZID (string),,,,,,,,,,,,,,,,,,,,,,,,,,
4,MGene,Gene ENTREZID (string),,,,,,,,,Gene Symbol (string),,,NCBI scientifc organism name (string),NCBI taxonomy id (string),,,,,,,,,,,,,
5,MethylationRegion,Identifier for base pair range (string),,,,,,,,,Name of methylation region (string),,,,,Distance to feature (int),Indicates whether the region overlaps with an exon (boolean),Indicates whether the region overlaps with the promoter (boolean),Name or number of the chromosome (string),Starting base pair position (int),Indicates whether the region overlaps with an intron (boolean),Ending base pair position (int),,,,,,
6,Mission,Identifier for mission (string),,,,,,,,,Name of the study (string),,,,,,,,,,,,End date of mission (date),Type of flight program (string),Sponsor of the mission (string),Start date of mission (date),,
7,Study,NASA Open Science Data Repository ID (string),,,,,,,,,Name of the study (string),,,NCBI scientific organism name (string),NCBI taxonomy ID (string),,,,,,,,,,,,Type of project (string),Title of project (string)


### Metagraph <a class="anchor" id="Metagraph"></a>
The metagraph shows the node labels and relationship types of the KG. Click on a node to display the node metadata.

In [6]:
query = """
MATCH p=(:MetaNode)-->(:MetaNode) RETURN p
"""
subgraph1 = graph.run(query).to_subgraph()

In [15]:
# If the layout isn’t ideal, rerun this cell.
# Use the mouse to drag nodes, pan or zoom the canvas, and adjust positions.
# Click a node to view its properties.
widget1 = neo4j_utils.draw_graph(subgraph1, stylesheet)
widget1.layout.height = "1014px"
widget1.set_layout(name="cola", padding=00, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=15000)
display(widget1)

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…

### Number of Nodes

In [8]:
query = """
MATCH (n) RETURN COUNT(n);
"""
print(f"Total number of nodes: {graph.evaluate(query)}")

Total number of nodes: 98123


In [9]:
query = """
MATCH (n) RETURN labels(n)[0] AS Node, COUNT(n) AS Count
ORDER BY Count DESC;
"""
graph.run(query).to_data_frame()

,Node,Count
0,MGene,57789
1,Gene,28291
2,Assay,6185
3,MethylationRegion,5663
4,Study,125
5,Anatomy,33
6,Mission,22
7,MetaNode,8
8,CellType,7


### Number of relationships

In [10]:
query = """
MATCH (n1)-[r]->(n2)
WITH TYPE(r)        AS Relationship,
     count(r)       AS Count,
     LABELS(n1)[0]  AS Node1,
     LABELS(n2)[0]  AS Node2
RETURN Node1, Relationship, Node2, Count
ORDER BY Count DESC;
"""
graph.run(query).to_data_frame()

,Node1,Relationship,Node2,Count
0,Assay,MEASURED_ASmMG,MGene,25878862
1,MGene,IS_ORTHOLOG_MGiG,Gene,56761
2,Assay,MEASURED_ASmMR,MethylationRegion,9555
3,Study,PERFORMED_SpAS,Assay,6185
4,Assay,INVESTIGATED_ASiA,Anatomy,6071
5,MGene,METHYLATED_IN_MGmMR,MethylationRegion,5694
6,Assay,INVESTIGATED_ASiCT,CellType,1134
7,Mission,CONDUCTED_MIcS,Study,74
8,MetaNode,MetaRelationship,MetaNode,8


## Fulltext Search <a class="anchor" id="Fulltext Search"></a>

### Full text query by keyword or phrase
A full text query returns all nodes that match the text query ([Query Syntax](https://lucene.apache.org/core/5_5_0/queryparser/org/apache/lucene/queryparser/classic/package-summary.html#Overview)). For exact matches, enclose the phrase in double quotes, e.g., ```'"eye"'```.

[Learn more about full-text searches.](https://graphaware.com/neo4j/2019/01/11/neo4j-full-text-search-deep-dive.html)

In [11]:
phrase = '"eye"'
top_n = 5 # show top n hits

In [12]:
query = """
CALL db.index.fulltext.queryNodes("fulltext", $phrase) YIELD node, score
RETURN node.identifier AS identifier, LABELS(node)[0] AS type, node;
"""
graph.run(query, phrase=phrase).to_data_frame().head(top_n)

,identifier,type,node
0,OSD-162-ccb721824557b1437385bd84ede20971,Assay,"{'identifier': 'OSD-162-ccb721824557b1437385bd84ede20971', 'material_name_1': 'eye', 'technology': 'RNA Sequencing (RNA-Seq)', 'material_name_2': 'eye', 'measurement': 'transcription profiling', 'factors_2': ['Ground Control'], 'material_2': 'eye', 'factors_1': ['Basal Control'], 'material_1': 'eye', 'name': 'OSD-162_transcription-profiling_rna-sequencing-(rna-seq)_illumina', 'material_id_1': 'UBERON:0000970', 'material_id_2': 'UBERON:0000970'}"
1,OSD-162-5a50e2cbc19079f36169127026802387,Assay,"{'identifier': 'OSD-162-5a50e2cbc19079f36169127026802387', 'material_name_1': 'eye', 'technology': 'RNA Sequencing (RNA-Seq)', 'material_name_2': 'eye', 'measurement': 'transcription profiling', 'factors_2': ['Space Flight'], 'material_2': 'eye', 'factors_1': ['Basal Control'], 'material_1': 'eye', 'name': 'OSD-162_transcription-profiling_rna-sequencing-(rna-seq)_illumina', 'material_id_1': 'UBERON:0000970', 'material_id_2': 'UBERON:0000970'}"
2,OSD-162-b087b97d7b5d2326e98bcb570a4e75ed,Assay,"{'identifier': 'OSD-162-b087b97d7b5d2326e98bcb570a4e75ed', 'material_name_1': 'eye', 'technology': 'RNA Sequencing (RNA-Seq)', 'material_name_2': 'eye', 'measurement': 'transcription profiling', 'factors_2': ['Ground Control'], 'material_2': 'eye', 'factors_1': ['Space Flight'], 'material_1': 'eye', 'name': 'OSD-162_transcription-profiling_rna-sequencing-(rna-seq)_illumina', 'material_id_1': 'UBERON:0000970', 'material_id_2': 'UBERON:0000970'}"
3,OSD-162-c7b1cabb29d13649b49721b6ee82da0f,Assay,"{'identifier': 'OSD-162-c7b1cabb29d13649b49721b6ee82da0f', 'material_name_1': 'eye', 'technology': 'RNA Sequencing (RNA-Seq)', 'material_name_2': 'eye', 'measurement': 'transcription profiling', 'factors_2': ['Basal Control'], 'material_2': 'eye', 'factors_1': ['Space Flight'], 'material_1': 'eye', 'name': 'OSD-162_transcription-profiling_rna-sequencing-(rna-seq)_illumina', 'material_id_1': 'UBERON:0000970', 'material_id_2': 'UBERON:0000970'}"
4,OSD-162-66374d8e18123f6973011ef691f1051e,Assay,"{'identifier': 'OSD-162-66374d8e18123f6973011ef691f1051e', 'material_name_1': 'eye', 'technology': 'RNA Sequencing (RNA-Seq)', 'material_name_2': 'eye', 'measurement': 'transcription profiling', 'factors_2': ['Space Flight'], 'material_2': 'eye', 'factors_1': ['Ground Control'], 'material_1': 'eye', 'name': 'OSD-162_transcription-profiling_rna-sequencing-(rna-seq)_illumina', 'material_id_1': 'UBERON:0000970', 'material_id_2': 'UBERON:0000970'}"


### Full text query using boolean operators
The full text query supports a variety of query types, including fuzzy, proximity, and range queries, as well as boolean operators ([Query Syntax](https://lucene.apache.org/core/5_5_0/queryparser/org/apache/lucene/queryparser/classic/package-summary.html#Overview)). The following example uses a query with an ```AND``` operator.

In [13]:
phrase = 'liver AND "DNA methylation profiling"'

In [14]:
query = """
CALL db.index.fulltext.queryNodes("fulltext", $phrase) YIELD node, score
RETURN node.identifier AS identifier, LABELS(node)[0] AS type, node, score
ORDER BY type;
"""
graph.run(query, phrase=phrase).to_data_frame()

,identifier,type,node,score
0,OSD-47-58ae5316b6c6291b4443efe11ddae112,Assay,"{'identifier': 'OSD-47-58ae5316b6c6291b4443efe11ddae112', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Ground Control'], 'material_2': 'Liver', 'factors_1': ['Basal Control'], 'material_1': 'Liver', 'name': 'OSD-47_dna-methylation-profiling_whole-genome-bisulfite-sequencing_illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
1,OSD-48-3d158da18c813ee1e06fc78453096a60,Assay,"{'identifier': 'OSD-48-3d158da18c813ee1e06fc78453096a60', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Space Flight', 'Upon euthanasia'], 'material_2': 'Liver', 'factors_1': ['Space Flight', 'Carcass'], 'material_1': 'Liver', 'name': 'OSD-48_dna-methylation-profiling_whole-genome-bisulfite-sequencing_Illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
2,OSD-48-5b94439e60f5c8ced57094bb2cd89965,Assay,"{'identifier': 'OSD-48-5b94439e60f5c8ced57094bb2cd89965', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Space Flight', 'Upon euthanasia'], 'material_2': 'Liver', 'factors_1': ['Ground Control', 'Upon euthanasia'], 'material_1': 'Liver', 'name': 'OSD-48_dna-methylation-profiling_whole-genome-bisulfite-sequencing_Illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
3,OSD-48-7f5d42ddd6a00c57482ebc9bce77b8a6,Assay,"{'identifier': 'OSD-48-7f5d42ddd6a00c57482ebc9bce77b8a6', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Space Flight', 'Carcass'], 'material_2': 'Liver', 'factors_1': ['Ground Control', 'Upon euthanasia'], 'material_1': 'Liver', 'name': 'OSD-48_dna-methylation-profiling_whole-genome-bisulfite-sequencing_Illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
4,OSD-48-e9056e3fbb849f81b553c7af1a1247ec,Assay,"{'identifier': 'OSD-48-e9056e3fbb849f81b553c7af1a1247ec', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Space Flight', 'Upon euthanasia'], 'material_2': 'Liver', 'factors_1': ['Ground Control', 'Carcass'], 'material_1': 'Liver', 'name': 'OSD-48_dna-methylation-profiling_whole-genome-bisulfite-sequencing_Illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
5,OSD-48-8910fc6d923eaf220fbbb06b7b96f274,Assay,"{'identifier': 'OSD-48-8910fc6d923eaf220fbbb06b7b96f274', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Space Flight', 'Carcass'], 'material_2': 'Liver', 'factors_1': ['Ground Control', 'Carcass'], 'material_1': 'Liver', 'name': 'OSD-48_dna-methylation-profiling_whole-genome-bisulfite-sequencing_Illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
6,OSD-48-0006b3853240c98c53d393f715dd0d54,Assay,"{'identifier': 'OSD-48-0006b3853240c98c53d393f715dd0d54', 'material_name_1': 'liver', 'technology': 'Whole Genome Bisulfite Sequencing', 'material_name_2': 'liver', 'measurement': 'DNA methylation profiling', 'factors_2': ['Ground Control', 'Upon euthanasia'], 'material_2': 'Liver', 'factors_1': ['Ground Control', 'Carcass'], 'material_1': 'Liver', 'name': 'OSD-48_dna-methylation-profiling_whole-genome-bisulfite-sequencing_Illumina', 'material_id_1': 'UBERON:0002107', 'material_id_2': 'UBERON:0002107'}",8.922503
7,OSD-47-d2d25640688118e0d80f292eb04ba995,Assa